In [ ]:
import os
import sys
import scipy.io
import imageio
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from PIL import Image
from nst_utils import *
import numpy as np
import tensorflow.compat.v1 as tf
import pprint
%matplotlib inline

In [ ]:
# Disable tensorflow v2 and reset default graph
tf.disable_v2_behavior()
tf.reset_default_graph()

In [ ]:
pp = pprint.PrettyPrinter(indent=4)
model = load_vgg_model("model_data/imagenet-vgg-verydeep-19.mat")
pp.pprint(model)

In [ ]:
content_image = imageio.imread("images/louvre.jpg")
imshow(content_image)

In [ ]:
def compute_content_cost(a_C, a_G):
    """
    a_C -- tensor of dimension (1, n_H, n_W, n_C), hidden layer activations representing content of the image C
    a_G -- tensor of dimension (1, n_H, n_W, n_C), hidden layer activations representing content of the image G
    """
    m, n_H, n_W, n_C = a_G.get_shape().as_list()
    a_C_unrolled = tf.reshape(a_C, shape=[m, -1, n_C])
    a_G_unrolled = tf.reshape(a_G, shape=[m, -1, n_C])
    J_content = 1 / (4 * n_H * n_W * n_C) * tf.reduce_sum(tf.square(tf.subtract(a_C_unrolled, a_G_unrolled)))
    return J_content

In [ ]:
style_image = imageio.imread("images/monet_800600.jpg")
imshow(style_image)

In [ ]:
def gram_matrix(A):
    GA = tf.matmul(A, tf.transpose(A))
    return GA

In [ ]:
def compute_layer_style_cost(a_S, a_G):
    m, n_H, n_W, n_C = a_G.get_shape().as_list()
    a_S = tf.transpose(tf.reshape(a_S, shape=[-1, n_C]), perm=[1, 0])
    a_G = tf.transpose(tf.reshape(a_G, shape=[-1, n_C]), perm=[1, 0])
    GS = gram_matrix(a_S)
    GG = gram_matrix(a_G)
    J_style_layer = 1 / (4 * n_C**2 * (n_H * n_W)**2) * tf.reduce_sum(tf.square(tf.subtract(GS, GG)))
    return J_style_layer

In [ ]:
STYLE_LAYERS = [
    ('conv1_1', 0.2),
    ('conv2_1', 0.2),
    ('conv3_1', 0.2),
    ('conv4_1', 0.2),
    ('conv5_1', 0.2)
]

In [ ]:
def compute_style_cost(model, STYLE_LAYERS):
    J_style = 0
    for layer_name, coeff in STYLE_LAYERS:
        out = model[layer_name]
        # Set a_S to be the hidden layer activation from they layer we have selected, by running the session on out
        a_S = sess.run(out)
        # Set a_G to be the hidden layer activation from the same layer. Here, a_G references model[layer_name]
        # and isn't evaluated yet. Latter in the code, we'll assign the image G as the model input, so that
        # when we run the session, this will be the activations drawn from the approprivate layer, with G as input. 
        a_G = out
        J_style_layer = compute_layer_style_cost(a_S, a_G)
        J_style += coeff * J_style_layer
    return J_style

In [ ]:
def total_cost(J_content, J_style, alpha=10, beta=40):
    J = alpha * J_content + beta * J_style
    return J

In [ ]:
tf.reset_default_graph()
sess = tf.InteractiveSession()

In [ ]:
content_image = imageio.imread("images/louvre_small.jpg")
content_image = reshape_and_normalize_image(content_image)

In [ ]:
style_image = imageio.imread("images/monet.jpg")
style_image = reshape_and_normalize_image(style_image)

In [ ]:
generated_image = generate_noise_image(content_image)
imshow(generated_image[0])

In [ ]:
model = load_vgg_model("model_data/imagenet-vgg-verydeep-19.mat")

In [ ]:
# Assign the content image to be the input of the VGG model
sess.run(model['input'].assign(content_image))
# select the output tensor of layer conv4_2
out = model['conv4_2']
# Set a_C to be the hidden layer activation from the layer we have selected
a_C = sess.run(out)
# Set a_G to be the hidden layer activation from same layer. Here, a_G references model['conv4_2'] 
# and isn't evaluated yet. Later in the code, we'll assign the image G as the model input, so that
# when we run the session, this will be the activations drawn from the appropriate layer, with G as input.
a_G = out
J_content = compute_content_cost(a_C, a_G)

In [ ]:
sess.run(model['input'].assign(style_image))
J_style = compute_style_cost(model, STYLE_LAYERS)

In [ ]:
J = total_cost(J_content, J_style, alpha=10, beta=40)

In [ ]:
optimizer = tf.train.AdamOptimizer(2.0)
train_step = optimizer.minimize(J)

In [ ]:
def model_nn(sess, input_image, num_interations=200):
    sess.run(tf.global_variables_initializer())
    # Run the noisy input image (initial generated image) through the model. Use assign()
    sess.run(model["input"].assign(input_image))

    for i in range(num_interations):
        # Run the session on the train_step to minimize total cost
        sess.run(train_step)
        # Compute the generated image by rinning the session on the current model['input']
        generated_image = sess.run(model['input'])
        
        if i%20 == 0:
            Jt, Jc, Js = sess.run([J, J_content, J_style])
            print("Iteration " + str(i) + " :")
            print("total cost = " + str(Jt))
            print("content cost = " + str(Jc))
            print("style cost = " + str(Js))
    
    save_image('images/generated_image.jpg', generated_image)
    return generated_image

In [ ]:
model_nn(sess, generated_image)